setting up the environment

In [1]:
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')


    basepath = '/content/drive/MyDrive'

else:
    basepath = '/home/harpreet/Insync/google_drive_shaannoor/data'


Mounted at /content/drive


Load Libraries

In [3]:
pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
# Importing PyTorch library for tensor computations and neural network modules
import torch
import torch.nn as nn

# For working with textual data vocabularies and for displaying model summaries

# General-purpose Python libraries for random number generation and numerical operations
import random
import numpy as np

# Utilities for efficient serialization/deserialization of Python objects and for element tallying
import joblib
from collections import Counter

# For creating lightweight attribute classes and for partial function application
from functools import partial

# For filesystem path handling, generating and displaying confusion matrices, and date-time manipulations
from pathlib import Path
from sklearn.metrics import confusion_matrix
from datetime import datetime

# For plotting and visualization
import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib inline

### NEW ##########################
# imports from Huggingface ecosystem
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import PreTrainedModel, PretrainedConfig
from transformers import TrainingArguments, Trainer
from datasets import Dataset
import evaluate

# wandb library
import wandb

In [12]:
from sklearn.metrics import accuracy_score, f1_score

In [5]:
# CHANGE FOLDERS TO WHERE YOU WANT TO SAVE DATA AND MODELS
base_folder = Path(basepath)
data_folder = base_folder/'datasets/HW6/Datafolder'

Load dataset

In [124]:
import pandas as pd
train_df = pd.read_csv(data_folder/'train.csv')
test_df = pd.read_csv(data_folder/'test.csv')
sample_submission = pd.read_csv(data_folder/'sample_submission.csv')

Data cleaning

In [8]:
pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 14.2 MB/s eta 0:00:00


In [125]:
# Define a function to clean the tweets
import re
import emoji

def clean_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet)
    tweet = re.sub(r'@\w+', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tweet = re.sub(r'\d+', '', tweet)
    tweet = re.sub(r'[^\w\s]', '', tweet)
    tweet = emoji.demojize(tweet, delimiters=(" ", " "))
    return tweet

train_df['Cleaned_Tweet'] = train_df['Tweet'].apply(clean_tweet)
test_df['Cleaned_Tweet'] = test_df['Tweet'].apply(clean_tweet)

In [126]:
# Preprocess labels in test_df to be integers (all 0s)
for emotion in ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']:
    test_df[emotion] = 0

In [127]:
# Keep only the 'Tweet' (as 'text') and 'label' columns
train_df = train_df[['ID', 'Cleaned_Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']].rename(columns={'Cleaned_Tweet': 'Tweet'})
test_df = test_df[['ID', 'Cleaned_Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']].rename(columns={'Cleaned_Tweet': 'Tweet'})


Tokenizer

In [128]:
# Define Custom Tokenizer with updated max_length
class CustomTokenizer:
    def __init__(self, max_length=130):  # Updated to 130
        self.max_length = max_length
        self.char2id = {"[PAD]": 0, "[UNK]": 1}
        self.id2char = {0: "[PAD]", 1: "[UNK]"}
        self.vocab_size = 2

    def build_vocab(self, texts):
        for text in texts:
            for char in text:
                if char not in self.char2id:
                    self.char2id[char] = self.vocab_size
                    self.id2char[self.vocab_size] = char
                    self.vocab_size += 1

    def tokenize(self, text):
        tokens = [self.char2id.get(char, self.char2id["[UNK]"]) for char in text]
        if len(tokens) > self.max_length:
            tokens = tokens[:self.max_length]
        else:
            tokens += [self.char2id["[PAD]"]] * (self.max_length - len(tokens))
        return tokens

In [129]:
# Initialize tokenizer and build vocabulary
tokenizer = CustomTokenizer(max_length=130)
all_texts = pd.concat([train_df['Tweet'], test_df['Tweet']], ignore_index=True)
tokenizer.build_vocab(all_texts)

Huggingface dataset

In [130]:
# Convert train and test data to Hugging Face Datasets with float labels
def preprocess_data(df, is_train=True):
    texts = df['Tweet'].tolist()
    input_ids = [tokenizer.tokenize(text) for text in texts]
    if is_train:
        labels = df.iloc[:, 2:].values.astype(float).tolist()  # Convert labels to float
        return {"input_ids": input_ids, "labels": labels}
    else:
        return {"input_ids": input_ids, "ID": df['ID'].tolist()}

In [131]:
train_dataset = Dataset.from_dict(preprocess_data(train_df, is_train=True))
test_dataset = Dataset.from_dict(preprocess_data(test_df, is_train=False))

Model

In [132]:
# Define the Model with explicit loss function handling
class SimpleClassifier(nn.Module):
    def __init__(self, input_dim=130, num_labels=train_df.shape[1] - 2):
        super(SimpleClassifier, self).__init__()
        self.embedding = nn.Embedding(256, 128)
        self.fc1 = nn.Linear(130 * 128, 64)
        self.fc2 = nn.Linear(64, num_labels)
        self.loss_fn = nn.BCEWithLogitsLoss()  # Use BCEWithLogitsLoss for multi-label classification

    def forward(self, input_ids, labels=None):
        x = self.embedding(input_ids)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        logits = self.fc2(x)  # Outputs raw logits without sigmoid

        # Compute loss if labels are provided (for training)
        if labels is not None:
            loss = self.loss_fn(logits, labels)
            return {"loss": loss, "logits": logits}
        else:
            return {"logits": logits}  # Only logits for inference

In [133]:
# Instantiate the model
model = SimpleClassifier(input_dim=130, num_labels=train_df.shape[1] - 2)

Compute metrics

In [134]:
# Define metric function for evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = (pred.predictions > 0.5).astype(int)
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='macro')
    return {'accuracy': accuracy, 'f1': f1}

Trainer

In [135]:
# Set up TrainingArguments and Trainer
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    save_strategy="epoch",
)

# Split train_dataset into training and validation sets using train_test_split directly
split_dataset = train_dataset.train_test_split(test_size=0.1)  # 10% for validation

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset['train'],
    eval_dataset=split_dataset['test'],  # Use validation dataset for evaluation
    tokenizer=None,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [137]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.382600,0.494639,0.051746,0.088273
2,0.344500,0.509640,0.055627,0.127829
3,0.310300,0.512032,0.056921,0.133307
4,0.297600,0.520010,0.062096,0.140951
5,0.283500,0.522727,0.056921,0.136143


TrainOutput(global_step=8690, training_loss=0.3176752255069921, metrics={'train_runtime': 35.1812, 'train_samples_per_second': 987.887, 'train_steps_per_second': 247.007, 'total_flos': 0.0, 'train_loss': 0.3176752255069921, 'epoch': 5.0})

Prediction on test and submission

In [139]:
# Predict on the test set
predictions = trainer.predict(test_dataset)
test_preds = (predictions.predictions > 0.5).astype(int)

# Prepare submission file using sample_submission format
submission = sample_submission[['ID']].copy()
submission[['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']] = pd.DataFrame(test_preds, index=submission.index)

# Save the submission file
submission.to_csv('submission_B.csv', index=False)

In [140]:
from google.colab import files
# Download the saved submission file
files.download("submission_B.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>